In [21]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns

# Set style for publication-quality figures
plt.rcParams['font.size'] = 12
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['axes.linewidth'] = 1.5
sns.set_palette("husl")

GOOD_UNC_COLOR="#4d8ec3"
BAD_UNC_COLOR="#e49c30"

def create_cp_auroc_plot():
    """
    Create uncertainty separation plot (correct vs incorrect predictions)
    """
    fig, ax = plt.subplots(figsize=(6, 4))
    
    # Generate synthetic data for correct and incorrect predictions
    np.random.seed(42)
    
    # Good model: clear separation
    correct_unc = np.random.beta(2, 5, 700) * 0.5  # Lower uncertainty
    incorrect_unc = np.random.beta(4, 3, 300) * 0.6 + 0.1  # Higher uncertainty
    
    # Create histogram
    bins = np.linspace(0, 1, 30)
    ax.hist(correct_unc, bins=bins, alpha=0.7, label='Correct Predictions', 
            color=GOOD_UNC_COLOR, edgecolor='black', linewidth=1.2)
    ax.hist(incorrect_unc, bins=bins, alpha=0.7, label='Incorrect Predictions', 
            color=BAD_UNC_COLOR, edgecolor='black', linewidth=1.2)
    
    ax.set_xlabel('Model Uncertainty (Softmax Response)', fontsize=14, fontweight='bold')
    ax.set_ylabel('density', fontsize=14, fontweight='bold')
    ax.set_title('Uncertainty Separability', fontsize=16, fontweight='bold', pad=20)
    ax.legend(loc='upper right', fontsize=11, frameon=True, shadow=True)
    ax.grid(True, alpha=0.3, linestyle='--', axis='y')
    
    # Add separation indicator
    ax.axvline(x=0.3, color='black', linestyle='--', linewidth=2, alpha=0.5)
    ax.text(0.3, ax.get_ylim()[1] * 0.9, 'Decision\nThreshold', 
            ha='center', fontsize=10, style='italic')
    
    plt.tight_layout()
    return fig

def create_selective_classification_plot():
    """
    Create selective classification curve (risk vs coverage) - AURC
    """
    fig, ax = plt.subplots(figsize=(6, 4))
    
    # Coverage from 0 to 100%
    coverage = np.linspace(0, 100, 100)
    
    # Base model accuracy
    base_acc = 75
    
    # Model with good uncertainty: can improve accuracy by abstaining
    good_unc_acc = base_acc + (100 - base_acc) * (1 - coverage/100)**0.5
    
    # Model with poor uncertainty: barely improves
    poor_unc_acc = base_acc + (100 - base_acc) * (1 - coverage/100)**2.5
    
    # Convert to risk (100 - accuracy)
    good_unc_risk = 100 - good_unc_acc
    poor_unc_risk = 100 - poor_unc_acc
    
    # Random selection (oracle baseline)
    random_risk = np.ones_like(coverage) * (100 - base_acc)
    
    ax.plot(coverage, good_unc_risk, linewidth=3, label='Good Uncertainty', 
            color=GOOD_UNC_COLOR, marker='o', markevery=10, markersize=6)
    ax.plot(coverage, poor_unc_risk, linewidth=3, label='Poor Uncertainty', 
            color=BAD_UNC_COLOR, marker='s', markevery=10, markersize=6)
    ax.plot(coverage, random_risk, 'k--', linewidth=2, label='No Selection (Baseline)', alpha=0.7)
    
    ax.set_xlabel('Coverage (% of predictions made)', fontsize=14, fontweight='bold')
    ax.set_ylabel('Risk (% error)', fontsize=14, fontweight='bold')
    ax.set_title('Risk-Coverage Tradeoff', fontsize=16, fontweight='bold', pad=20)
    ax.legend(loc='lower right', fontsize=11, frameon=False, shadow=False)
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.set_xlim(0, 110)
    ax.set_ylim(0, 30)
    
    # Highlight area between curves - gain in AURC
    ax.fill_between(coverage, good_unc_risk, poor_unc_risk, alpha=0.2, color='gold')
    ax.text(50, 15, 'Gain in AURC', ha='center', fontsize=10, 
            style='italic', color='darkgoldenrod', fontweight='bold')
    
    # Label error rate at coverage=100
    error_rate = 100 - base_acc
    ax.plot(100, error_rate, 'ko', markersize=8, zorder=5)
    ax.text(105, error_rate + 1, f'Error Rate\n({error_rate:.0f}%)', ha='right', fontsize=10, 
            fontweight='bold')
    
    # Add horizontal line at risk=0.5 to show SAC@95 gain
    risk_threshold = 5
    # Find coverage where each curve reaches this risk level
    idx_good = np.argmin(np.abs(good_unc_risk - risk_threshold))
    idx_poor = np.argmin(np.abs(poor_unc_risk - risk_threshold))
    cov_good = coverage[idx_good]
    cov_poor = coverage[idx_poor]
    
    # Draw horizontal line between the two curves at this risk level
    ax.plot([cov_poor, cov_good], [risk_threshold, risk_threshold], 
            '-', linewidth=2.5, alpha=0.7, zorder=2,color="gray")
    ax.plot([cov_poor, cov_good], [risk_threshold, risk_threshold], 
            'o', markersize=6, zorder=2,color="gray")
    
    # Label the gain in SAC@95
    mid_cov = (cov_good + cov_poor) / 2
    ax.text(mid_cov+0.8, risk_threshold + 0.4, f'Gain in SAC@{risk_threshold:.1f}', 
            ha='center', fontsize=9, fontweight='bold', color='gray')
    
    # Add annotation
    ax.text(40, 27, 'Abstain on uncertain examples → lower risk',
            fontsize=11, ha='center', style='italic',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    return fig


def main():
    """
    Generate all figures for the poster
    """
    print("Generating uncertainty visualization figures...")
    
    # Create output directory
    import os
    os.makedirs('uncertainty_figures', exist_ok=True)

    print("1. Creating calibration plot...")
    fig1 = create_calibration_plot()
    fig1.savefig('uncertainty_figures/calibration.png', dpi=300, bbox_inches='tight')
    fig1.savefig('uncertainty_figures/calibration.pdf', bbox_inches='tight')
    plt.close(fig1)

    print("2. Creating CP-AUROC plot...")
    fig2 = create_cp_auroc_plot()
    fig2.savefig('uncertainty_figures/cp_auroc.png', dpi=300, bbox_inches='tight')
    fig2.savefig('uncertainty_figures/cp_auroc.pdf', bbox_inches='tight')
    plt.close(fig2)
    
    print("3. Creating selective classification plot...")
    fig3 = create_selective_classification_plot()
    fig3.savefig('uncertainty_figures/selective_classification.png', dpi=300, bbox_inches='tight')
    fig3.savefig('uncertainty_figures/selective_classification.pdf', bbox_inches='tight')
    plt.close(fig3)
    
    print("\nAll figures saved to 'uncertainty_figures/' directory")
    print("Generated both PNG (for poster) and PDF (for high-quality printing)")

if __name__ == "__main__":
    main()

Generating uncertainty visualization figures...
1. Creating calibration plot...
2. Creating CP-AUROC plot...
3. Creating selective classification plot...

All figures saved to 'uncertainty_figures/' directory
Generated both PNG (for poster) and PDF (for high-quality printing)


In [20]:
def create_calibration_plot():
    """
    Create calibration plot comparing well-calibrated vs overconfident model
    """
    fig, ax = plt.subplots(figsize=(6, 4))
    
    # Set random seed for reproducibility
    np.random.seed(42)
    
    # Create 10 bins
    n_bins = 10
    bin_edges = np.linspace(0, 1, n_bins + 1)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    # Well-calibrated model: predicted confidence matches actual accuracy with some noise
    calibrated_confidence = bin_centers
    calibrated_accuracy = bin_centers + np.random.normal(0, 0.03, n_bins)  # Random noise
    calibrated_accuracy = np.clip(calibrated_accuracy, 0, 1)
    
    # Overconfident model: actual accuracy is always lower than predicted confidence with noise
    overconfident_confidence = bin_centers
    # Create systematic overconfidence - worse at higher confidence
    overconfidence_gap = 0.05 + 0.3 * bin_centers  # Gap increases with confidence
    overconfident_accuracy = bin_centers - overconfidence_gap + np.random.normal(0, 0.05, n_bins)
    overconfident_accuracy = np.clip(overconfident_accuracy, 0, 1)
    
    # Plot perfect calibration line first (so it's in the background)
    ax.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Perfect Calibration', alpha=0.7, zorder=1)
    
    # Plot calibrated model with transparency
    ax.plot(calibrated_confidence, calibrated_accuracy, linewidth=3, 
            label='Calibrated', color=GOOD_UNC_COLOR, marker='o', markersize=8, 
            alpha=0.8, zorder=3)
    
    # Plot overconfident model with transparency
    ax.plot(overconfident_confidence, overconfident_accuracy, linewidth=3, 
            label='Overconfident', color=BAD_UNC_COLOR, marker='s', markersize=8, 
            alpha=0.8, zorder=2)
    
    ax.set_xlabel('Predicted Confidence', fontsize=14, fontweight='bold')
    ax.set_ylabel('Actual Accuracy', fontsize=14, fontweight='bold')
    ax.set_title('Calibration', fontsize=16, fontweight='bold', pad=20)
    ax.legend(loc='upper left', fontsize=11, frameon=True, shadow=True)
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    
    plt.tight_layout()
    return fig